In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request as request
import time
import re
import urllib.parse

In [ ]:
def writeOrderInfo(text):
    fh = open('订单信息.txt', 'a')
    fh.write(text+'\n')
    fh.close()

## 第三方接口

In [ ]:
# 从非小号抓取最新价格（USDT）
def getFxhPrice(symbol):
    symbolList = {'BTC_USDT': 'bitcoin', 'ETH_USDT': 'ethereum'}
    url = 'https://www.feixiaohao.com/currencies/'+symbolList[symbol]
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
    requestss = request.Request(url,
                                method='GET',
                                headers=headers)
    data = request.urlopen(requestss).read().decode('utf-8')
    soup = BeautifulSoup(data,'html.parser')
    priceText = soup.find_all('div',attrs={'class':'sub'})[4].get_text()
    priceStr = ""
    for i in re.findall(r"\d+", re.findall(r"\$(.*)\xa0\xa0", priceText)[0]):
        priceStr+=i
    priceInt = int(priceStr)
    return priceInt

## 自定义CoinBig接口

In [ ]:
# 获取交易数据，买一卖一取均价
def getTradeData(browser, symbol, timeDelay):
    if browser.current_url != 'https://www.coinbig.com/pl/trade/'+symbol:
        browser.get('https://www.coinbig.com/pl/trade/'+symbol)
        time.sleep(timeDelay)
    pageSource = browser.page_source
    time.sleep(timeDelay)
    pageSource = browser.page_source
    soup=BeautifulSoup(pageSource,'lxml')
    # soup.find('div', class_='H_eigh W_list')
    #买一
    pricebuy = float(soup.find('div', class_='H_eight W_list').find_all('span', class_='bl b1_img')[0].text)
    #卖一
    pricesell = float(soup.find('div', class_='H_eigh W_list').find_all('span', class_='bl b1_img')[-1].text)
    return round((pricesell+pricebuy)/2, 4)

In [ ]:
# 切换当前交易方向
def orderDirExchange(browser, orderDir):
#     orderDir = '买入'
    orderDirDict = {'买入': 'money01', '卖出': 'money02'}
    browser.find_element_by_xpath("//a[@aria-controls='{0}']".format(orderDirDict[orderDir])).click()

In [ ]:
# 挂单
def trade(browser, symbol, orderDir, orderPrice, orderAmount, timeDelay):
    # 确认在当前交易***“品种”***及方向
    while 1:
        if browser.current_url != 'https://www.coinbig.com/pl/trade/'+order:
            browser.get('https://www.coinbig.com/pl/trade/'+order)
        pageSource = browser.page_source
        soup=BeautifulSoup(pageSource,'lxml')
        if soup.find('li', class_='activess').text[:2] != orderDir:
            orderDirExchange(browser, orderDir)
            time.sleep(timeDelay)
        else:
            break
            
    elementPrice = browser.find_element_by_xpath("//input[@id='txtPrice']")
    elementPrice.clear()
    elementPrice.send_keys(str(orderPrice))
    
    if orderAmount == 0:
        # orderAmount为0时，买入卖出全部
        browser.find_element_by_xpath("//div[@style='left: 100%;']").click()
    else:
        elementAmount = browser.find_element_by_xpath("//input[@id='txtSnumber']")
        elementAmount.clear()
        elementAmount.send_keys(str(orderAmount))
    try:
        browser.find_element_by_xpath("//input[@value='{0}']".format(orderDir)).click()
        code = 0
    except:
        code = 1
        
    return code

In [ ]:
# 查询订单数
def orderInfo(browser):
    pageSource = browser.page_source
    soup=BeautifulSoup(pageSource,'lxml')
    orderCount = len(soup.find_all('div', class_='table-div table-div-sm text-center clearfix P_evenchild'))
    return orderCount

## 自动交易挖矿程序实现

#### 1.login()登陆函数

In [ ]:
# 登陆并转到交易品种页面
def login(CbAccount, CbPassword, symbol, timeDelay):
    browser = webdriver.Chrome()
    browser.get('https://www.coinbig.com/userlogin.html?lang=zh_CN')
    # 将浏览器最大化
    browser.maximize_window()
    # 自动输入用户名密码
    time.sleep(timeDelay*3)
    elementAcc = browser.find_element_by_xpath("//input[@placeholder='登录账号']")
    elementAcc.send_keys(CbAccount)
    elementPwd = browser.find_element_by_xpath("//input[@placeholder='登录密码']")
    elementPwd.send_keys(CbPassword)
    # 模拟点击登录
    browser.find_element_by_xpath("//input[@value='登录']").click()
    time.sleep(timeDelay)
    # 转到交易品种页面
    while 1:
        if browser.current_url == 'https://www.coinbig.com/pl/trade' or browser.current_url == 'https://www.coinbig.com/pl/trade/BTC_USDT':
            browser.get('https://www.coinbig.com/pl/trade/'+symbol)
            time.sleep(timeDelay)
            break
    return browser

#### 2.luPrice()价格设置及检查函数，及子函数setupCheck()

In [ ]:
def setupCheck(run, lowerPrice, upperPrice, cbPrice):
    fxhPrice = getFxhPrice(symbol)
    if lowerPrice>=upperPrice:
        print("警告！自动交易价格上下限设置错误")
        run = 1
    elif abs(upperPrice-lowerPrice)>0.001*(upperPrice+lowerPrice)/2:
        isCont = input("警告！当前上下限价差超过了千一，继续执行请输入“YES”：")
        if isCont!="YES":
            run = 1
    if abs((lowerPrice+upperPrice)/2-cbPrice)>0.001*cbPrice and run==0:
        isCont = input("警告！当前上下限均价和CB现价差距过大，成交率会降低，继续执行请输入“YES”：")
        if isCont!="YES":
            run = 1
    if abs((lowerPrice+upperPrice)/2-fxhPrice)>0.01*fxhPrice and run==0:
        isCont = input("警告！当前上下限均价和非小号行情价差距过大，继续执行请输入“YES”：")
        if isCont!="YES":
            run = 1
    return run

In [ ]:
def luPrice(browser, symbol, timeDelay):
    # 输入价格符合检查后getCom置0，开始自动交易，否则无限循环
    run = 0
    while(run==0):
        fxhPrice = getFxhPrice(symbol)
        cbPrice = getTradeData(browser, symbol, timeDelay)                 
        print("------------------------------------\n当前非小号价格   ："+str(fxhPrice)+" USDT\nCoinBig盘口均价："+str(cbPrice)+
              " USDT\n建议最大开单区间【"+str(round(cbPrice*0.9997, 4))+
              ","+str(round(cbPrice*1.0003, 4))+"】\n------------------------------------")

        lowerPrice = float(input("请输入自动交易价格下限："))
        upperPrice = float(input("请输入自动交易价格上限："))
        orderAmount = float(input("请输入单次挂单数量,为0则是按当前账号余额进行全买全卖操作"))
        
        # 对设置的lowerPrice，upperPrice，orderAmount进行检查
        run = setupCheck(run, lowerPrice, upperPrice, cbPrice)
            
        # 循环调整, run=0则运行正常，置1跳出，进入交易部分
        if run == 0:
            run = 1
        else:
            run = 0
            
    return lowerPrice, upperPrice, orderAmount

#### 3.orderProgram()下单函数，及子函数getOrderPrice()，orderScan()

In [ ]:
def getOrderPrice(MODE, browser, symbol, upperPrice, lowerPrice, timeDelay):
    if MODE == 0:
        orderPrice = getTradeData(browser, symbol, timeDelay)
    elif MODE == 1:
        if orderDirCode == 0:
            # 买入价格为设置价格上限
            orderPrice = upperPrice
        else:
            # 卖出价格为设置价格下限
            orderPrice = lowerPrice

In [ ]:
def orderScan(browser, symbol, lowerPrice, upperPrice, orderDirCode, timeDelay):
    orderScanCounter = 0
    while 1:
        orderCount = orderInfo(browser)
        #长时间未成交撤销后，重按最新价挂单
        orderScanCounter +=1
        if orderCount == 0:
            writeOrderInfo("订单已成交\n")
            print("订单已成交！")
            # 订单完成，改变交易方向,初始化挂单量
            orderDirCode = (orderDirCode+1)%2
            break
        elif orderCount>1 or orderScanCounter>150:
            writeOrderInfo("订单量异常或长时间未完全成交\n")
            err = 1
            while int(err) != 0:
                err = input("订单量异常或长时间未完全成交，请处理完后输入“0”继续：")
            # 重新初始化设置
            lowerPrice, upperPrice, orderAmount = luPrice(browser, symbol, timeDelay)
            newOrderDirCode = int(input("请输入下面开始成交方向，0为买，1为卖："))
            while newOrderDirCode!=0 and newOrderDirCode!=1:
                newOrderDirCode = int(input("输入错误，请重新输入："))
            orderDirCode = newOrderDirCode
            break
    return lowerPrice, upperPrice, orderAmount, orderDirCode

In [ ]:
def orderProgram(MODE, browser, symbol, orderDirList, orderDirCode, upperPrice, lowerPrice, orderAmount, timeDelay):
    while 1:
        counter = 0
        try:
            # 根据MODE，获取下单价
            orderPrice = getOrderPrice(MODE, browser, symbol, upperPrice, lowerPrice, timeDelay)
            
            # 当前价格满足则挂单(限价单)
            orderDir = orderDirList[orderDirCode]
            
            if lowerPrice<=orderPrice<=upperPrice:
                writeOrderInfo("【挂单】"+orderDir+" | 价格："+str(orderPrice)+" | 数量："+str(orderAmount)+
                               "       Time:"+str(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime())))
                tradedata = trade(browser, symbol, orderDir, orderPrice, orderAmount)
                if tradedata==1:
                    # 挂单失败
                    writeOrderInfo("挂单失败")
                    err = 1
                    while int(err) != 0:
                        err = input("挂单失败，请检查！异常处理完毕后请输入“0”继续：")
                else:
                    print("[方向："+orderDir+"|价格："+str(orderPrice)+"|数量："+str(orderAmount)+"]挂单成功!")
                    writeOrderInfo("挂单成功！")
                    time.sleep(timeDelay)
                    # 下单后循环监控该订单情况，完全成交后才继续
                    lowerPrice, upperPrice, orderAmount, orderDirCode = orderScan(browser, symbol, lowerPrice, \
                                                                                  upperPrice, orderDirCode, timeDelay)
            else:
                counter+=1
                if counter>150:
                    err = 1
                    while int(err) != 0:
                        err = input("当前市场价长时间不符合开单区间，请修改开单价区间或稍后输入“0”继续：")
                    lowerPrice, upperPrice, orderAmount = luPrice(browser, symbol, timeDelay)
                else:
                    print("当前价格["+str(orderPrice)+"]不满足开单条件，继续监控")
                    time.sleep(timeDelay)
        except:
            err = 1
            while int(err) != 0:
                err = input("程序运行出错，请检查！异常处理完毕后请输入“0”继续：")
            newOrderDirCode = int(input("请输入下面开始成交方向，0为买，1为卖："))
            while newOrderDirCode!=0 and newOrderDirCode!=1:
                newOrderDirCode = int(input("输入错误，请重新输入："))
            orderDirCode = newOrderDirCode

#### 4.主函数

In [ ]:
if __name__ == "__main__":
    """
    MODE为模式设置，有0和1两种，设为0时，下单价格为CB最新成交价，设为1时，下单价格为自定义的上下限价格。
    （MODE=1能提高订单成交速度，但矿损会扩大，选择此方式的话请谨慎选择上下限价格）
    """
    MODE = 0
    
    # CB账号配置
    CbAccount = "***"
    CbPassword = "***"
    
    # 设置交易品种，0为BTC，1为ETH
    symbolList = ['BTC_USDT', 'ETH_USDT']
    symbolCode = 0
    symbol = symbolList[symbolCode]
    
    # 设置交易方向，orderDirList全为“buy”则一直挂买单，全为“sell”则一直挂卖单
    orderDirList = ["买入", "卖出"]
    # "0"则先买后卖，1则先卖后买
    orderDirCode = 0
    
    # 全局时间延长设置（单位：s），在网络不稳定时进行调整
    timeDelay = 2
    
    # 登陆
    browser = login(CbAccount, CbPassword, symbol, timeDelay)
    # 价格设置及检查
    lowerPrice, upperPrice, orderAmount = luPrice(browser, symbol, timeDelay)

    # 下单主程序
    orderProgram(MODE, browser, symbol, orderDirList, orderDirCode, upperPrice, lowerPrice, orderAmount, timeDelay)
                    
    input("程序结束，请输入任意字符关闭！")